<h1><b><i>Todas Tabelas - Script</i></b></h1>

In [12]:
import os
import pandas as pd
import xlsxwriter    

In [13]:
# Define o caminho do arquivo Excel
file_path = 'Planilha Monitoramento_PPI - Carregamento - BI 2025 - 6.xlsx'

In [14]:
# Pasta onde estão os arquivos de origem
pasta_dados = 'Dados Gerados/'

In [15]:
# Lista com os caminhos dos arquivos a serem combinados
arquivos_e_sheets = {
    'INFORMAÇÕES (1).xlsx': 'INFORMAÇÕES',
    'PER (1).xlsx': 'PER', 
    'PLAN_EXEC ATÉ ANO ANTERIOR (24).xlsx': 'PLAN_EXEC (ATÉ 2024)',
    'META(2025).xlsx': 'META(2025)',
    'iNEXECUTADO (pendências até 24).xlsx': 'INEXECUTADO ATÉ 2024 Pendências', 
    'À EXECUTAR (2026 DIANTE).xlsx': 'A EXECUTAR',
    'META(2026).xlsx': 'META(2026)',
    'META(2027).xlsx': 'META(2027)'
}

In [16]:
# Inicializar um DataFrame vazio para concatenar todos os dados
df_todas_tabelas = pd.DataFrame()

In [17]:
# Criar o arquivo Excel
with pd.ExcelWriter(file_path, engine='xlsxwriter', engine_kwargs={'options': {'strings_to_numbers': True}}) as writer:
    for arquivo, nome_sheet in arquivos_e_sheets.items():
        caminho_arquivo = os.path.join(pasta_dados, arquivo)
        try:
            df = pd.read_excel(caminho_arquivo)
            df['PLANILHA'] = nome_sheet
            df.to_excel(writer, sheet_name=nome_sheet, index=False)
            df_todas_tabelas = pd.concat([df_todas_tabelas, df], ignore_index=True)
        except FileNotFoundError:
            print(f'Arquivo não encontrado: {caminho_arquivo}')

    # Processamento final
    df_todas_tabelas.index = range(1, len(df_todas_tabelas) + 1)
    df_todas_tabelas = df_todas_tabelas.reset_index()
    df_todas_tabelas.rename(columns={'index': 'ID-ÚNICO-TT'}, inplace=True)

    if 'Atributo.2' in df_todas_tabelas.columns and 'Valor' in df_todas_tabelas.columns:
        df_todas_tabelas_mod = df_todas_tabelas.pivot_table(
            index='ID-ÚNICO-TT', columns='Atributo.2', values='Valor', aggfunc='first'
        ).reset_index()

        df_final = pd.merge(df_todas_tabelas, df_todas_tabelas_mod, on='ID-ÚNICO-TT', how='outer')
        df_final['EMPREENDIMENTO'] = df_final['EMPREENDIMENTO'].replace('CONCESSÃO VIA SUL', 'VIA SUL')

        df_final.drop(columns=['Atributo.2', 'Valor'], errors='ignore', inplace=True)
        df_final.rename(columns={'Atributo.3': 'Atributo.2'}, inplace=True)

        colunas_para_zerar = ['(km)%', '(km)% EXEC', '(km)% PLAN', 'Ext. (km)', 'FINANCEIRO (R$)', 'FINANCEIRO PLAN (R$)',
                              'PERCENTUAL (%)', 'PERCENTUAL (%) EXEC', 'PERCENTUAL (%) PLAN', 'km (f)', 'km (i)']
        for col in colunas_para_zerar:
            if col in df_final.columns:
                df_final[col] = df_final[col].fillna(0)

        df_final.to_excel(writer, sheet_name='Todas Tabelas', index=False)
    else:
        print('Não Deu Certo :(')

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_13380\3074325620.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final[col] = df_final[col].fillna(0)


<h1><b><i>'Todas Tabelas - Script'</i></b></h1>
<h2><b><i>1. Descrição do Processo</i></b></h2>
<p>O script realiza a combinação de várias planilhas de diferentes arquivos Excel em um único arquivo, utilizando o Pandas e o ExcelWriter do XlsxWriter. Ele também organiza os dados, preenche valores ausentes e aplica pivotagens conforme necessário.</p>

<h2><b><i>2. Etapas do Processamento</i></b></h2>

<h3><b><i>2.1. Definição dos Arquivos e Sheets</i></b></h3>
<p>O script começa definindo os arquivos e suas respectivas planilhas que precisam ser carregados e combinados. A lista 'arquivos_e_sheets' contém os nomes dos arquivos Excel e as planilhas a serem lidas de cada arquivo:</p>
<pre>
arquivos_e_sheets = {
'INFORMAÇÕES (1).xlsx': 'INFORMAÇÕES',
'PER (1).xlsx': 'PER',
'PLAN_EXEC ATÉ ANO ANTERIOR (24).xlsx': 'PLAN_EXEC (ATÉ 2024)',
'META(2025).xlsx': 'META(2025)',
'iNEXECUTADO (pendências até 24).xlsx': 'INEXECUTADO ATÉ 2024 Pendências',
'À EXECUTAR (2026 DIANTE).xlsx': 'A EXECUTAR',
'META(2026).xlsx': 'META(2026)',
'META(2027).xlsx': 'META(2027)'
}
</pre>

<h3><b><i>2.2. Inicialização do DataFrame</i></b></h3>
<p>O script inicializa um DataFrame vazio, <code>df_todas_tabelas</code>, que será preenchido com os dados de cada arquivo:</p>
<pre>
df_todas_tabelas = pd.DataFrame()
</pre>

<h3><b><i>2.3. Leitura dos Arquivos e Criação das Sheets</i></b></h3>
<p>Utiliza-se o ExcelWriter para abrir um novo arquivo e escrever os dados de cada arquivo de entrada nas suas respectivas planilhas. O nome da planilha é adicionado como uma nova coluna em cada DataFrame:</p>
<pre>
with pd.ExcelWriter(file_path, engine='xlsxwriter', engine_kwargs={'options': {'strings_to_numbers': True}}) as writer:
for arquivo, nome_sheet in arquivos_e_sheets.items():
df = pd.read_excel(arquivo)
df['PLANILHA'] = nome_sheet  # Adiciona a coluna 'PLANILHA' com o nome do arquivo
df.to_excel(writer, sheet_name=nome_sheet, index=False)
df_todas_tabelas = pd.concat([df_todas_tabelas, df], ignore_index=True)
</pre>

<h3><b><i>2.4. Adição de Índice Único</i></b></h3>
<p>O índice do DataFrame resultante é modificado para garantir que cada linha tenha um valor único. A coluna do índice é renomeada para 'ID-ÚNICO-TT':</p>
<pre>
df_todas_tabelas.index = range(1, len(df_todas_tabelas) + 1)
df_todas_tabelas = df_todas_tabelas.reset_index()
df_todas_tabelas.rename(columns={'index': 'ID-ÚNICO-TT'}, inplace=True)
</pre>

<h3><b><i>2.5. Pivotagem dos Dados</i></b></h3>
<p>Se as colunas 'Atributo.2' e 'Valor' estiverem presentes, o script cria uma tabela pivô para reestruturar os dados:</p>
<pre>
if 'Atributo.2' in df_todas_tabelas.columns and 'Valor' in df_todas_tabelas.columns:
df_todas_tabelas_mod = df_todas_tabelas.pivot_table(index=['ID-ÚNICO-TT'], columns='Atributo.2', values='Valor', aggfunc='first').reset_index()
</pre>

<h3><b><i>2.6. Combinação dos Dados</i></b></h3>
<p>O script combina os dados originais com os dados pivotados para criar uma única tabela final:</p>
<pre>
df_final = pd.merge(df_todas_tabelas, df_todas_tabelas_mod, on='ID-ÚNICO-TT', how='outer')
</pre>

<h3><b><i>2.7. Substituição de Valores e Preenchimento de Nulos</i></b></h3>
<p>Na coluna 'EMPREENDIMENTO', é feita uma substituição específica de valor, e os valores ausentes nas colunas numéricas são preenchidos com 0:</p>
<pre>
df_final['EMPREENDIMENTO'] = df_final['EMPREENDIMENTO'].replace('CONCESSÃO VIA SUL', 'VIA SUL')
df_final[['(km)%', '(km)% EXEC', '(km)% PLAN', 'Ext. (km)', 'FINANCEIRO (R$)', 'FINANCEIRO PLAN (R$)',
'PERCENTUAL (%)', 'PERCENTUAL (%) EXEC', 'PERCENTUAL (%) PLAN',
'km (f)', 'km (i)']] = df_final[['(km)%', '(km)% EXEC', '(km)% PLAN', 'Ext. (km)', 'FINANCEIRO (R$)', 'FINANCEIRO PLAN (R$)',
'PERCENTUAL (%)', 'PERCENTUAL (%) EXEC', 'PERCENTUAL (%) PLAN',
'km (f)', 'km (i)']].fillna(0)
</pre>

<h3><b><i>2.8. Finalização e Salvamento</i></b></h3>
<p>Finalmente, a tabela final é salva em uma nova aba chamada 'Todas Tabelas'. Caso a estrutura não seja encontrada, uma mensagem de erro será exibida:</p>
<pre>
df_final.to_excel(writer, sheet_name='Todas Tabelas', index=False)  
</pre>

<h3><b><i>2.9. Mensagem de Erro</i></b></h3>
<p>Caso as colunas necessárias ('Atributo.2' e 'Valor') não existam no DataFrame, uma mensagem de erro é impressa no console:</p>
<pre>
print('Não Deu Certo :( ')
</pre>

<h2><b><i>3. Conclusão</i></b></h2>
<p>O script combina e transforma os dados de múltiplos arquivos e planilhas em um único arquivo Excel, criando uma nova aba com os dados processados. As informações são limpas, reorganizadas, e preparadas para análises subsequentes.</p>

---